In [5]:
import pandas as pd
from sodapy import Socrata
import os
from dotenv import load_dotenv
load_dotenv('.env')

TOKEN_APP= os.getenv("TOKEN_SODAPY")
socrata_dataset_identifier = os.getenv("DATASET_ID")

client = Socrata("www.datos.gov.co",
               TOKEN_APP )## Contraseña de ingreso a la plataforma de datos abiertos

In [6]:
client.timeout = 10000

In [22]:
Query = """
select 
    id_contrato, nombre_entidad, departamento, descripcion_del_proceso, valor_del_contrato,fecha_de_firma
where
    fecha_de_firma >= '2024-01-01'
limit
10000000
"""

results = client.get(socrata_dataset_identifier, query=Query)

#results


In [23]:
results_df = pd.DataFrame.from_records(results)
results_df

,id_contrato,nombre_entidad,departamento,descripcion_del_proceso,valor_del_contrato,fecha_de_firma
0,CO1.PCCNTR.3113043,MUNICIPIO DE SAN EDUARDO,Boyacá,"AUNAR ESFUERZOS TÉCNICOS, ADMINISTRATIVOS Y\nF...",44154631,2024-01-01T00:00:00.000
1,CO1.PCCNTR.5663788,Ese Centro de Salud San Miguel,Nariño,PRESTACION DE SERVICIOS PROFESIONALES DE MEDIC...,5500000,2024-01-01T00:00:00.000
2,CO1.PCCNTR.5663795,Ese Centro de Salud San Miguel,Nariño,PRESTACION DE SERVICIOS DE AUXILIAR DE ENFERME...,1800000,2024-01-01T00:00:00.000
3,CO1.PCCNTR.5663796,Ese Centro de Salud San Miguel,Nariño,PRESTACION DE SERVICIOS PROFESIONALES DE PSICO...,3000000,2024-01-01T00:00:00.000
4,CO1.PCCNTR.5664637,Ese Centro de Salud San Miguel,Nariño,PRESTACION DE SERVICIOS DE AUXILIAR DE ENFERME...,1800000,2024-01-01T00:00:00.000
...,...,...,...,...,...,...
591568,CO1.PCCNTR.6530005,CENTRO DE SALUD CON CAMAS CANTAGALLO L.,Bolívar,PRESTACIÓN DE SERVICIOS PROFESIONALES EN LA SI...,16800000,2024-07-14T00:00:00.000
591569,CO1.PCCNTR.6530108,INSTITUTO DE DEPORTE Y RECREACIÓN DE CALDAS INDEC,Antioquia,PRESTACIÓN DE SERVICIOS DE UN FORMADOR PARA AP...,11077000,2024-07-14T00:00:00.000
591570,CO1.PCCNTR.6530403,ALCALDIA LOCAL DE ANTONIO NARIÑO,Distrito Capital de Bogotá,PRESTAR SERVICIOS PROFESIONALES ESPECIALIZADOS...,40000000,2024-07-14T00:00:00.000
591571,CO1.PCCNTR.6530344,INSTITUTO MUNICIPAL DE DEPORTES EDUCACION FISI...,Antioquia,Contrato de Prestación de servicios y Apoyo a ...,15594000,2024-07-15T00:00:00.000


## Exploración Inicial

En este notebook se realizará una exploración inicial de los datos para entender su estructura y contenido. Descargamos los datos de todos los contratos de 2024 en SECOP y vamos a hacer un análisis simple de estos datos.

In [24]:
### Veamos los tipos de datos de las columnas

results_df.dtypes

id_contrato                object
nombre_entidad             object
departamento               object
descripcion_del_proceso    object
valor_del_contrato         object
fecha_de_firma             object
dtype: object

In [28]:
dataset_contrato = results_df[results_df['id_contrato'] == 'CO1.PCCNTR.6530403'].T
dataset_contrato

,591570
id_contrato,CO1.PCCNTR.6530403
nombre_entidad,ALCALDIA LOCAL DE ANTONIO NARIÑO
departamento,Distrito Capital de Bogotá
descripcion_del_proceso,PRESTAR SERVICIOS PROFESIONALES ESPECIALIZADOS...
valor_del_contrato,"40,000,000.00"
fecha_de_firma,2024-07-14T00:00:00.000


In [25]:
results_df['valor_del_contrato'] = results_df['valor_del_contrato'].astype(float)
results_df['fecha_de_firma'] = pd.to_datetime(results_df['fecha_de_firma'])

In [26]:
### Veamos la distribución de los contratos por departamento

results_df['departamento'].value_counts()

departamento
Distrito Capital de Bogotá                  219772
Antioquia                                    56148
Valle del Cauca                              54932
Cundinamarca                                 30501
Santander                                    24030
Boyacá                                       17183
Bolívar                                      16773
Tolima                                       15439
Atlántico                                    14476
Norte de Santander                           12913
Meta                                         12252
Magdalena                                    11879
Caldas                                       10816
Quindío                                      10439
Risaralda                                    10260
Nariño                                        9482
Huila                                         9310
Cauca                                         9154
Casanare                                      8317
Cesar             

In [27]:
### Veamos el valor total de los contratos por departamento

pd.set_option('display.float_format', '{:,.2f}'.format)

results_df.groupby('departamento')['valor_del_contrato'].sum()

departamento
Amazonas                                      163,257,285,287.00
Antioquia                                   8,789,765,836,092.00
Arauca                                        127,793,006,782.00
Atlántico                                   2,719,149,745,225.00
Bolívar                                     2,318,813,412,082.00
Boyacá                                      1,171,342,730,356.00
Caldas                                      1,285,042,793,646.00
Caquetá                                       343,507,914,044.00
Casanare                                      614,262,742,909.00
Cauca                                       1,354,264,702,603.00
Cesar                                         979,298,636,314.00
Chocó                                         812,726,055,441.00
Cundinamarca                                2,016,428,643,334.00
Córdoba                                       897,104,889,606.00
Distrito Capital de Bogotá                 26,204,710,588,358.00
Guainía     